## Connect to IDR

In [1]:
from omero.gateway import BlitzGateway
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


## Connect to IDR testing 

In [2]:
HOST = 'ws://idr-testing.openmicroscopy.org/omero-ws'
conn_testing = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn_testing.connect())
conn_testing.c.enableKeepAlive(60)

True


## Load an image

We load information about the image but **not** the binary data. An image is a 5D-object (XYZCT).

In [3]:
image_id = 9036344

In [4]:
image = conn.getObject("Image", image_id)
print(image.getName())

1B_010618_Hsp104Htb1_Meiosis_01_47_R3D.dv


In [5]:
image_testing = conn_testing.getObject("Image", image_id)

In [6]:
print("Size Z: %s, Size C: %s, Size T: %s" % (image.getSizeZ(), image.getSizeC(), image.getSizeC()))

Size Z: 8, Size C: 2, Size T: 2


## Load the binary data

To access the binary data, we need to load the `pixels` object from the image and retrieve the plane(s)

In [7]:
def load_plane(image, z, c, t):
    pixels = image.getPrimaryPixels()
    return pixels.getPlane(z, c, t)

In [8]:
%%time
# Load from idr
plane = load_plane(image, 0, 0, 0)
print(plane.shape)

(1024, 1024)
CPU times: user 439 ms, sys: 114 ms, total: 552 ms
Wall time: 2.05 s


In [9]:
%%time
# Load from idr-testing
plane_testing = load_plane(image_testing, 0, 0, 0)

CPU times: user 119 ms, sys: 59.3 ms, total: 178 ms
Wall time: 3.29 s


To load multiple planes use ``pixels.getPlanes()``

In [10]:
zct_list = [(0, 0, 0), (1, 0, 0)]

In [11]:
def load_planes(image, zct_list):
    pixels = image.getPrimaryPixels()
    return pixels.getPlanes(zct_list)

In [12]:
%%time
# Load from idr
planes = load_planes(image, zct_list)

CPU times: user 233 µs, sys: 26 µs, total: 259 µs
Wall time: 285 µs


In [ ]:
%%time
# Load from idr-testing
planes_testing = load_planes(image_testing, zct_list)

In [5]:
def load_5D_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    #z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for c in range(size_c):  # all channels
            for z in range(size_z):  # get the Z-stack
                zct_list.append((z, c, t))

    # Load all the planes as YX numpy array
    return pixels.getPlanes(zct_list)

In [6]:
%%time
# Load from idr
planes = load_5D_array(image)

KeyboardInterrupt: 

In [ ]:
%%time
# Load from idr-testing
planes_testing = load_5D_array(image_testing)